In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("ColabSparkSession") \
    .getOrCreate()

# <font color="orange">Understanding Spark partition </font>

In [ ]:
Run below command to get block information about the file

hdfs fsck /user/training/spark-data/rating-small/ -blocks

### Create a Dataframe

In [ ]:
smallratingDF = spark.read.parquet("spark-data/rating-small")

In [ ]:
smallratingDF.rdd.getNumPartitions()

__Spark decided to read these small 601 files in 22 partition__

In [4]:
hugeratingDF = spark.read.parquet("spark-data/rating-huge")

In [ ]:
hugeratingDF.rdd.getNumPartitions()

__Spark decided to read these small 1 files in 2 partition__

In [ ]:
ratingDF = spark.read.option("header","true").csv("spark-data/ratings.csv")
ratingDF.rdd.getNumPartitions()

In [8]:
spark.conf.set("spark.sql.adaptive.enabled","true")
spark.conf.set("spark.sql.shuffle.partitions",200)
ratingDF.groupBy("rating").count().write.mode("overwrite").csv("spark-data/output/ratings")

__This time spark decided to read this 646MB file in 6 partitions__

<font color="sky blue">Question: How Spark deciding on number of partition? Is it random or based on some formula?</font>

 Calculation of Spark Partition depends on below properties:
        
- spark.default.parallelism (default: Total No. of CPU cores)
- spark.sql.files.maxPartitionBytes (default: 128 MB)
- spark.sql.files.openCostInBytes (default: 4 MB)


First it calculates bytePercore which is based on 

- sum of size of all files
- no of files
- total no of cores of executors (default.parallelism)
- spark.sql.files.openCostInBytes (default: 4 MB)

<font color="light blue">___bytesPerCore = (Sum of sizes of all data files + No. of files * openCostInBytes) / default.parallelism___</font>


Then we compare above calculated value with spark.sql.files.maxPartitionBytes (default: 128 MB),
whichever is minimum that becomes our partition size

<font color="light blue">maxSplitBytes = Minimum(maxPartitionBytes, bytesPerCore)</font>



### <font color="blue"> Assignment </font>

In How many Spark Partitions, below files will be read

54 parquet files, 63 MB each, all 3 config parameters at default, No. of core equal to 10

54 parquet files, 40 MB each, all 3 config parameters at default, No. of core equal to 10


54 parquet files, 40 MB each, maxPartitionBytes set to 88 MB, other two configs at default values., No. of core equal to 10